In [12]:
import pandas as pd

In [13]:
data = pd.read_csv('./messages_languages.csv', index_col=0)

In [14]:
data

,language,score,text,date
0,sw,0.999996,hi,2021-05-01T00:00:12
1,en,0.999997,do indicators work?,2021-05-01T00:01:13
2,en,0.999996,If you trade true gbp for gbp Fiat or vice Ver...,2021-05-01T00:01:50
3,en,0.999995,And how much is that fee?,2021-05-01T00:02:05
4,en,0.999998,If you’re a silver card holder,2021-05-01T00:02:12
...,...,...,...,...
49364,pt,0.857141,Yea I do,2021-05-15T23:56:33
49365,en,0.714282,I beleive it will tank.,2021-05-15T23:57:02
49366,en,0.999998,Are you worry that too many see that URL is a ...,2021-05-15T23:57:16
49367,en,0.999996,Dammit i want some fake giveaways too. Where i...,2021-05-15T23:58:56


In [15]:
data['words_count'] = data['text'].apply(lambda x: len(str(x).split()))

In [16]:
data = data[data['words_count'] >= 3]

In [17]:
data = data[data['language'] == 'en']

In [18]:
data = data[data['text'].str.contains('SHIB|DOGE') == True]

In [19]:
data

,language,score,text,date,words_count
1816,en,0.999996,"If DOGE is more worth than CRO for now, who kn...",2021-05-01T16:40:46,18
2036,en,0.999994,HOW ABOUT DOGE???,2021-05-01T19:21:28,3
3943,en,0.999999,Congratulations rue*****@*****com!\n🚀 You are ...,2021-05-02T21:16:47,41
5394,en,0.999997,Congratulations rue*****@*****com!\n🚀 You are ...,2021-05-03T12:16:38,41
5824,en,0.999997,DOGE coin is up by +16.39%🚀🚀🚀,2021-05-03T16:44:13,6
...,...,...,...,...,...
49091,en,0.999997,Is there a way I can see how much volume of SH...,2021-05-15T21:01:50,19
49092,en,0.999996,"From where to buy SHIB in India, any leads ple...",2021-05-15T21:04:06,11
49094,en,0.999997,SHIB is listed through the Cryptocom App 🙏,2021-05-15T21:08:02,8
49103,en,0.999996,Binance listed SHIB just this week.,2021-05-15T21:13:01,6


# NLTK

In [20]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\parsa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\parsa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
texts = pd.DataFrame(data[['text','date']]).reset_index().drop('index', axis=1)

In [22]:
texts

,text,date
0,"If DOGE is more worth than CRO for now, who kn...",2021-05-01T16:40:46
1,HOW ABOUT DOGE???,2021-05-01T19:21:28
2,Congratulations rue*****@*****com!\n🚀 You are ...,2021-05-02T21:16:47
3,Congratulations rue*****@*****com!\n🚀 You are ...,2021-05-03T12:16:38
4,DOGE coin is up by +16.39%🚀🚀🚀,2021-05-03T16:44:13
...,...,...
423,Is there a way I can see how much volume of SH...,2021-05-15T21:01:50
424,"From where to buy SHIB in India, any leads ple...",2021-05-15T21:04:06
425,SHIB is listed through the Cryptocom App 🙏,2021-05-15T21:08:02
426,Binance listed SHIB just this week.,2021-05-15T21:13:01


In [23]:
import string
import re
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [24]:
def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub("(@[A-Za-z0-9_]+)","", text)
    # remove numbers
    text_nonum = re.sub(r'\d+', '', text)
    # remove punctuations and convert characters to lower case
    text_nopunct = "".join([char.lower() for char in text_nonum if char not in string.punctuation])
    text_nopunct = re.sub(r'’', '', text_nopunct)

    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    # substitute multiple whitespace with single whitespace
    # Also, removes leading and trailing whitespaces
    text_no_doublespace = re.sub('\s+', ' ', emoji_pattern.sub(r'', text_nopunct)).strip()
    return text_no_doublespace

In [25]:
texts['clean_text']= texts['text'].apply(lambda x:clean_text(x))
texts

,text,date,clean_text
0,"If DOGE is more worth than CRO for now, who kn...",2021-05-01T16:40:46,if doge is more worth than cro for now who kno...
1,HOW ABOUT DOGE???,2021-05-01T19:21:28,how about doge
2,Congratulations rue*****@*****com!\n🚀 You are ...,2021-05-02T21:16:47,congratulations ruecom you are the weekly luck...
3,Congratulations rue*****@*****com!\n🚀 You are ...,2021-05-03T12:16:38,congratulations ruecom you are the weekly luck...
4,DOGE coin is up by +16.39%🚀🚀🚀,2021-05-03T16:44:13,doge coin is up by
...,...,...,...
423,Is there a way I can see how much volume of SH...,2021-05-15T21:01:50,is there a way i can see how much volume of sh...
424,"From where to buy SHIB in India, any leads ple...",2021-05-15T21:04:06,from where to buy shib in india any leads please
425,SHIB is listed through the Cryptocom App 🙏,2021-05-15T21:08:02,shib is listed through the cryptocom app
426,Binance listed SHIB just this week.,2021-05-15T21:13:01,binance listed shib just this week


In [26]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\parsa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\parsa\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [27]:
tokenized_text = []

In [28]:
for text in texts['clean_text'].tolist():
    tokenized_text.append(nltk.word_tokenize(text))

In [29]:
from nltk.tag import pos_tag
print(pos_tag(tokenized_text[6]))

[('eth', 'RB'), ('doge', 'NN'), ('bnb', 'NN'), ('leading', 'VBG'), ('the', 'DT'), ('market', 'NN'), ('making', 'VBG'), ('new', 'JJ'), ('all', 'DT'), ('time', 'NN'), ('highs', 'NNS')]


In [30]:
from nltk.stem.wordnet import WordNetLemmatizer

In [31]:
stopwords = nltk.corpus.stopwords.words("english")

In [32]:
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        token = lemmatizer.lemmatize(word, pos)
        lemmatized_sentence.append(token)
        # if len(token) > 0 and token not in stopwords:
        #     lemmatized_sentence.append(token)
    return lemmatized_sentence

In [33]:
print(lemmatize_sentence(tokenized_text[0]))

['if', 'doge', 'be', 'more', 'worth', 'than', 'cro', 'for', 'now', 'who', 'know', 'what', 'cro', 'will', 'do', 'in', 'the', 'future']


In [34]:
tokens = []
for tt in tokenized_text:
    tokens.append(lemmatize_sentence(tt))

tokens

[['if',
  'doge',
  'be',
  'more',
  'worth',
  'than',
  'cro',
  'for',
  'now',
  'who',
  'know',
  'what',
  'cro',
  'will',
  'do',
  'in',
  'the',
  'future'],
 ['how', 'about', 'doge'],
 ['congratulation',
  'ruecom',
  'you',
  'be',
  'the',
  'weekly',
  'lucky',
  'draw',
  'winner',
  'of',
  'our',
  'doge',
  'treat',
  'extravaganza',
  'you',
  'have',
  'win',
  'usd',
  'an',
  'exclusive',
  'doge',
  'nft',
  'check',
  'out',
  'the',
  'top',
  'winner',
  'here',
  'with',
  'the',
  'high',
  'number',
  'of',
  'doge',
  'point'],
 ['congratulation',
  'ruecom',
  'you',
  'be',
  'the',
  'weekly',
  'lucky',
  'draw',
  'winner',
  'of',
  'our',
  'doge',
  'treat',
  'extravaganza',
  'you',
  'have',
  'win',
  'usd',
  'an',
  'exclusive',
  'doge',
  'nft',
  'check',
  'out',
  'the',
  'top',
  'winner',
  'here',
  'with',
  'the',
  'high',
  'number',
  'of',
  'doge',
  'point'],
 ['doge', 'coin', 'be', 'up', 'by'],
 ['your',
  'do',
  'not',
 

In [35]:
texts['tokenized_clean_text'] = tokens

In [36]:
texts

,text,date,clean_text,tokenized_clean_text
0,"If DOGE is more worth than CRO for now, who kn...",2021-05-01T16:40:46,if doge is more worth than cro for now who kno...,"[if, doge, be, more, worth, than, cro, for, no..."
1,HOW ABOUT DOGE???,2021-05-01T19:21:28,how about doge,"[how, about, doge]"
2,Congratulations rue*****@*****com!\n🚀 You are ...,2021-05-02T21:16:47,congratulations ruecom you are the weekly luck...,"[congratulation, ruecom, you, be, the, weekly,..."
3,Congratulations rue*****@*****com!\n🚀 You are ...,2021-05-03T12:16:38,congratulations ruecom you are the weekly luck...,"[congratulation, ruecom, you, be, the, weekly,..."
4,DOGE coin is up by +16.39%🚀🚀🚀,2021-05-03T16:44:13,doge coin is up by,"[doge, coin, be, up, by]"
...,...,...,...,...
423,Is there a way I can see how much volume of SH...,2021-05-15T21:01:50,is there a way i can see how much volume of sh...,"[be, there, a, way, i, can, see, how, much, vo..."
424,"From where to buy SHIB in India, any leads ple...",2021-05-15T21:04:06,from where to buy shib in india any leads please,"[from, where, to, buy, shib, in, india, any, l..."
425,SHIB is listed through the Cryptocom App 🙏,2021-05-15T21:08:02,shib is listed through the cryptocom app,"[shib, be, list, through, the, cryptocom, app]"
426,Binance listed SHIB just this week.,2021-05-15T21:13:01,binance listed shib just this week,"[binance, list, shib, just, this, week]"


In [37]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

In [38]:
all_text_words = get_all_words(texts['tokenized_clean_text'].tolist())

In [39]:
from nltk import FreqDist

freq_dist_pos = FreqDist(all_text_words)
print(freq_dist_pos.most_common(10))

[('be', 292), ('the', 291), ('i', 280), ('to', 247), ('shib', 232), ('and', 170), ('buy', 157), ('shiba', 132), ('doge', 131), ('on', 130)]


In [40]:
texts['final_text'] = texts['tokenized_clean_text'].apply(lambda li: ' '.join(li))

In [42]:
texts

,text,date,clean_text,tokenized_clean_text,final_text
0,"If DOGE is more worth than CRO for now, who kn...",2021-05-01T16:40:46,if doge is more worth than cro for now who kno...,"[if, doge, be, more, worth, than, cro, for, no...",if doge be more worth than cro for now who kno...
1,HOW ABOUT DOGE???,2021-05-01T19:21:28,how about doge,"[how, about, doge]",how about doge
2,Congratulations rue*****@*****com!\n🚀 You are ...,2021-05-02T21:16:47,congratulations ruecom you are the weekly luck...,"[congratulation, ruecom, you, be, the, weekly,...",congratulation ruecom you be the weekly lucky ...
3,Congratulations rue*****@*****com!\n🚀 You are ...,2021-05-03T12:16:38,congratulations ruecom you are the weekly luck...,"[congratulation, ruecom, you, be, the, weekly,...",congratulation ruecom you be the weekly lucky ...
4,DOGE coin is up by +16.39%🚀🚀🚀,2021-05-03T16:44:13,doge coin is up by,"[doge, coin, be, up, by]",doge coin be up by
...,...,...,...,...,...
423,Is there a way I can see how much volume of SH...,2021-05-15T21:01:50,is there a way i can see how much volume of sh...,"[be, there, a, way, i, can, see, how, much, vo...",be there a way i can see how much volume of sh...
424,"From where to buy SHIB in India, any leads ple...",2021-05-15T21:04:06,from where to buy shib in india any leads please,"[from, where, to, buy, shib, in, india, any, l...",from where to buy shib in india any lead please
425,SHIB is listed through the Cryptocom App 🙏,2021-05-15T21:08:02,shib is listed through the cryptocom app,"[shib, be, list, through, the, cryptocom, app]",shib be list through the cryptocom app
426,Binance listed SHIB just this week.,2021-05-15T21:13:01,binance listed shib just this week,"[binance, list, shib, just, this, week]",binance list shib just this week


In [43]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\parsa\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [44]:
texts['sentiment'] = texts['final_text'].apply(lambda x: sid.polarity_scores(x))

In [45]:
texts

,text,date,clean_text,tokenized_clean_text,final_text,sentiment
0,"If DOGE is more worth than CRO for now, who kn...",2021-05-01T16:40:46,if doge is more worth than cro for now who kno...,"[if, doge, be, more, worth, than, cro, for, no...",if doge be more worth than cro for now who kno...,"{'neg': 0.0, 'neu': 0.886, 'pos': 0.114, 'comp..."
1,HOW ABOUT DOGE???,2021-05-01T19:21:28,how about doge,"[how, about, doge]",how about doge,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,Congratulations rue*****@*****com!\n🚀 You are ...,2021-05-02T21:16:47,congratulations ruecom you are the weekly luck...,"[congratulation, ruecom, you, be, the, weekly,...",congratulation ruecom you be the weekly lucky ...,"{'neg': 0.0, 'neu': 0.506, 'pos': 0.494, 'comp..."
3,Congratulations rue*****@*****com!\n🚀 You are ...,2021-05-03T12:16:38,congratulations ruecom you are the weekly luck...,"[congratulation, ruecom, you, be, the, weekly,...",congratulation ruecom you be the weekly lucky ...,"{'neg': 0.0, 'neu': 0.506, 'pos': 0.494, 'comp..."
4,DOGE coin is up by +16.39%🚀🚀🚀,2021-05-03T16:44:13,doge coin is up by,"[doge, coin, be, up, by]",doge coin be up by,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
...,...,...,...,...,...,...
423,Is there a way I can see how much volume of SH...,2021-05-15T21:01:50,is there a way i can see how much volume of sh...,"[be, there, a, way, i, can, see, how, much, vo...",be there a way i can see how much volume of sh...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
424,"From where to buy SHIB in India, any leads ple...",2021-05-15T21:04:06,from where to buy shib in india any leads please,"[from, where, to, buy, shib, in, india, any, l...",from where to buy shib in india any lead please,"{'neg': 0.0, 'neu': 0.796, 'pos': 0.204, 'comp..."
425,SHIB is listed through the Cryptocom App 🙏,2021-05-15T21:08:02,shib is listed through the cryptocom app,"[shib, be, list, through, the, cryptocom, app]",shib be list through the cryptocom app,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
426,Binance listed SHIB just this week.,2021-05-15T21:13:01,binance listed shib just this week,"[binance, list, shib, just, this, week]",binance list shib just this week,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [46]:
texts['date'] = pd.to_datetime(texts['date']).dt.date

In [47]:
texts

,text,date,clean_text,tokenized_clean_text,final_text,sentiment
0,"If DOGE is more worth than CRO for now, who kn...",2021-05-01,if doge is more worth than cro for now who kno...,"[if, doge, be, more, worth, than, cro, for, no...",if doge be more worth than cro for now who kno...,"{'neg': 0.0, 'neu': 0.886, 'pos': 0.114, 'comp..."
1,HOW ABOUT DOGE???,2021-05-01,how about doge,"[how, about, doge]",how about doge,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,Congratulations rue*****@*****com!\n🚀 You are ...,2021-05-02,congratulations ruecom you are the weekly luck...,"[congratulation, ruecom, you, be, the, weekly,...",congratulation ruecom you be the weekly lucky ...,"{'neg': 0.0, 'neu': 0.506, 'pos': 0.494, 'comp..."
3,Congratulations rue*****@*****com!\n🚀 You are ...,2021-05-03,congratulations ruecom you are the weekly luck...,"[congratulation, ruecom, you, be, the, weekly,...",congratulation ruecom you be the weekly lucky ...,"{'neg': 0.0, 'neu': 0.506, 'pos': 0.494, 'comp..."
4,DOGE coin is up by +16.39%🚀🚀🚀,2021-05-03,doge coin is up by,"[doge, coin, be, up, by]",doge coin be up by,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
...,...,...,...,...,...,...
423,Is there a way I can see how much volume of SH...,2021-05-15,is there a way i can see how much volume of sh...,"[be, there, a, way, i, can, see, how, much, vo...",be there a way i can see how much volume of sh...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
424,"From where to buy SHIB in India, any leads ple...",2021-05-15,from where to buy shib in india any leads please,"[from, where, to, buy, shib, in, india, any, l...",from where to buy shib in india any lead please,"{'neg': 0.0, 'neu': 0.796, 'pos': 0.204, 'comp..."
425,SHIB is listed through the Cryptocom App 🙏,2021-05-15,shib is listed through the cryptocom app,"[shib, be, list, through, the, cryptocom, app]",shib be list through the cryptocom app,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
426,Binance listed SHIB just this week.,2021-05-15,binance listed shib just this week,"[binance, list, shib, just, this, week]",binance list shib just this week,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [48]:
plot_df = texts.groupby('date').size().reset_index(name='count')

In [49]:
import plotly.graph_objects as go

In [62]:
fig = go.Figure([go.Scatter(x=plot_df['date'], y=plot_df['count'])])
fig.show()

In [53]:
texts['sentiment_score'] = texts['sentiment'].apply(lambda obj: obj['pos'] - obj['neg'])

In [54]:
texts

,text,date,clean_text,tokenized_clean_text,final_text,sentiment,sentiment_score
0,"If DOGE is more worth than CRO for now, who kn...",2021-05-01,if doge is more worth than cro for now who kno...,"[if, doge, be, more, worth, than, cro, for, no...",if doge be more worth than cro for now who kno...,"{'neg': 0.0, 'neu': 0.886, 'pos': 0.114, 'comp...",0.114
1,HOW ABOUT DOGE???,2021-05-01,how about doge,"[how, about, doge]",how about doge,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000
2,Congratulations rue*****@*****com!\n🚀 You are ...,2021-05-02,congratulations ruecom you are the weekly luck...,"[congratulation, ruecom, you, be, the, weekly,...",congratulation ruecom you be the weekly lucky ...,"{'neg': 0.0, 'neu': 0.506, 'pos': 0.494, 'comp...",0.494
3,Congratulations rue*****@*****com!\n🚀 You are ...,2021-05-03,congratulations ruecom you are the weekly luck...,"[congratulation, ruecom, you, be, the, weekly,...",congratulation ruecom you be the weekly lucky ...,"{'neg': 0.0, 'neu': 0.506, 'pos': 0.494, 'comp...",0.494
4,DOGE coin is up by +16.39%🚀🚀🚀,2021-05-03,doge coin is up by,"[doge, coin, be, up, by]",doge coin be up by,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000
...,...,...,...,...,...,...,...
423,Is there a way I can see how much volume of SH...,2021-05-15,is there a way i can see how much volume of sh...,"[be, there, a, way, i, can, see, how, much, vo...",be there a way i can see how much volume of sh...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000
424,"From where to buy SHIB in India, any leads ple...",2021-05-15,from where to buy shib in india any leads please,"[from, where, to, buy, shib, in, india, any, l...",from where to buy shib in india any lead please,"{'neg': 0.0, 'neu': 0.796, 'pos': 0.204, 'comp...",0.204
425,SHIB is listed through the Cryptocom App 🙏,2021-05-15,shib is listed through the cryptocom app,"[shib, be, list, through, the, cryptocom, app]",shib be list through the cryptocom app,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000
426,Binance listed SHIB just this week.,2021-05-15,binance listed shib just this week,"[binance, list, shib, just, this, week]",binance list shib just this week,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000


In [60]:
plot_df2 = texts.groupby('date').mean().reset_index()
plot_df2

,date,sentiment_score
0,2021-05-01,0.057000
1,2021-05-02,0.494000
2,2021-05-03,0.164667
3,2021-05-04,-0.087667
4,2021-05-05,0.051500
5,2021-05-06,0.092167
6,2021-05-07,0.090444
7,2021-05-08,0.039642
8,2021-05-09,0.022077
9,2021-05-10,0.007580


In [61]:
fig = go.Figure([go.Scatter(x=plot_df2['date'], y=plot_df2['sentiment_score'])])
fig.show()